In [2]:
# miscellaneous
import matplotlib.pyplot as plt

from glob import glob #library that helps us search for files
import scipy
import random
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
from scipy.special import inv_boxcox
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

# hyperparameter tuning
from hyperopt import hp
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials, partial

# preprocessing
from category_encoders import OneHotEncoder,TargetEncoder,OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer, PowerTransformer,LabelEncoder, MaxAbsScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score,cross_val_predict, KFold, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer, PowerTransformer,LabelEncoder, MaxAbsScaler, RobustScaler

# models
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression,BayesianRidge, ElasticNet, Lasso
from sklearn.dummy import DummyClassifier
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier, VotingClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import clone
import lightgbm as lgb

# metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
from sklearn.metrics import log_loss, confusion_matrix, classification_report, precision_recall_curve
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, PrecisionRecallDisplay
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.utils.validation import check_is_fitted

# feature selection / data sampling
from sklearn.feature_selection import RFE, SelectKBest, f_classif, SelectFromModel, VarianceThreshold
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.inspection import permutation_importance

# saving model
import pickle,joblib
import boto3

# data settings
pd.pandas.set_option('display.max_rows',None)
pd.pandas.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)


from sklearn import set_config
set_config(transform_output = 'pandas')

import warnings
warnings.simplefilter(action="ignore")


import json

# custom package
import importlib
import package
importlib.reload(package)

from package.data_retrieval.data_retrieval import s3_retrieval, write_to_s3
from package.eda.data_exploration import correlation
from package.training.model_training import model_evaluation, split_data_by_type
from package.preprocessing.data_preprocessing import create_pipeline, winsorize, percentile_imputer, random_sample_imputer, count_encoder 

# Read Config

In [3]:
config_path = "C:/Users/Oamen/OneDrive/Documents/DATA PROJECTS/Insurance_claim_prediction_porto/config.json"

In [13]:
# Opening JSON file
with open(config_path) as file:
    
    config = json.load(file)

In [14]:
config['Model Training'] = {}

config['Model Training']['Preprocessor'] = {}

# Read Data

In [15]:
# get access key
secret = pd.read_csv("C:/Users/Oamen/Downloads/jupyter_iam_accessKeys.csv")

In [16]:
# create S3 client object 
s3_client = boto3.client('s3',aws_access_key_id=secret['Access key ID'][0],aws_secret_access_key=secret['Secret access key'][0])

In [17]:
bucket_name = config['EDA']['bucket_name'] 

In [18]:
# retrieve data
full_train_df = s3_retrieval(s3_client, bucket_name, config['Model_Selection']['Input_data']['full_train'])

Retrieved object
Read bytes
Success!


In [19]:
#reduced_train_df = s3_retrieval(s3_client, bucket_name, reduced_train_new_key)

In [20]:
validation_df = s3_retrieval(s3_client, bucket_name, config['Model_Selection']['Input_data']['validation'])

Retrieved object
Read bytes
Success!


In [21]:
test_df = s3_retrieval(s3_client, bucket_name, config['Model_Selection']['Input_data']['test'])

Retrieved object
Read bytes
Success!


# Training Preprocessor

## Preprocessor - Experiment 1

1. Missing Values: Simple
2. Encoding: One-Hot
3. Transformation: Log
4. Outliers: Winsorize(0.95-0.05)
5. Scaling: StandardScaler
6. Resampling: SMOTE
7. Feature Selection: Enet Coefs

In [22]:
categorical_exp1, numerical_exp1, xtrain_exp1, ytrain_exp1 = split_data_by_type(full_train_df, config['EDA']['target'])

In [23]:
categorical_exp1, numerical_exp1, xvalidation_exp1, yvalidation_exp1 = split_data_by_type(validation_df, config['EDA']['target'])

In [24]:
num_pipeline_exp1 = Pipeline([
    ('transformer', FunctionTransformer(np.log1p)),
    ('imputer', SimpleImputer(strategy = 'median')),
    ('winsorize', FunctionTransformer(winsorize)),
    ('scaler', StandardScaler()),  
])

cat_pipeline_exp1 = Pipeline([
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('encoder', OneHotEncoder()),
    
])

In [25]:
pipeline_exp1 = create_pipeline(num_pipeline_exp1, numerical_exp1, cat_pipeline_exp1, categorical_exp1)

In [26]:
xtrain_exp1 = pipeline_exp1.fit_transform(xtrain_exp1)

In [27]:
xvalidation_exp1 = pipeline_exp1.transform(xvalidation_exp1)

In [28]:
# smote

smote = SMOTE()

In [29]:
xtrain_exp1, ytrain_exp1 = smote.fit_resample(xtrain_exp1, ytrain_exp1)

In [30]:
# feature selection - Enet Coefs

enet = ElasticNet(l1_ratio = 0.5, alpha = 0.01)

enet.fit(xtrain_exp1, ytrain_exp1)

ElasticNet(alpha=0.01)

In [31]:
imp_columns1 = dict(zip(xtrain_exp1.columns, enet.coef_))

In [35]:
selected_columns_exp1 = [column for column, value in imp_columns1.items() if abs(value) > 0]

len(selected_columns_exp1)

32

In [37]:
xtrain_exp1 = xtrain_exp1[selected_columns_exp1]

xvalidation_exp1 = xvalidation_exp1[selected_columns_exp1]

### Hyperparameter Tuning - Exp 1

#### Random Forest

In [56]:
# create the search space for an over fitting model

search_space = {
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'criterion': hp.choice('criterion', ['gini', 'entropy']),
    'max_depth': hp.quniform('max_depth', 2, 30, 1),
    'min_samples_split': hp.quniform('min_samples_split', 4, 30, 1),
    'min_weight_fraction_leaf': hp.uniform('min_weight_fraction_leaf', 0.0, 0.5),
    'max_leaf_nodes': hp.quniform('max_leaf_nodes',8,30,1),
    'warm_start': hp.choice('warm_start', [True, False]),
    'class_weight': hp.choice('class_weight', ['balanced', 'balanced_subsample']),
    'ccp_alpha': hp.uniform('ccp_alpha', 0.0, 0.9)
   
}

In [57]:
def objective(params, n_folds, x, y):
    
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    params['min_samples_split'] = int(params['min_samples_split'])
    params['max_leaf_nodes'] = int(params['max_leaf_nodes'])
    
#     params['min_weight_fraction_leaf'] = round(params['min_weight_fraction_leaf'], 2)
#     params['ccp_alpha'] = round(params['ccp_alpha'], 2)
    
    
    model = RandomForestClassifier(**params, random_state = 0)
    
    scores = cross_val_score(model, x, y, cv = n_folds, scoring = 'roc_auc', error_score='raise')
    
    max_score = max(scores)
    
    # to minimize
    loss = 1 - max_score
    
    return {'loss':loss,
           'params':params,
           'status': STATUS_OK}

In [ ]:
# optimize with the TPE algorithm
trials = Trials()
n_folds = 10

best = fmin(fn = partial(objective, n_folds = n_folds,
                         x = xtrain_exp1, y = ytrain_exp1),
           space = search_space, algo = tpe.suggest, max_evals = 16, trials = trials,
           )

  0%|                                                                           | 0/16 [00:00<?, ?trial/s, best loss=?]

In [ ]:
print(best)

#### LightGBM

In [ ]:
search_space = {
    'num_leaves': hp.quniform('num_leaves', 2,31,1),
    'max_depth': hp.quniform('max_depth', 2,31,1),
    'n_estimators': hp.quniform('n_estimators', 2,100,1),
    'min_child_samples': hp.quniform('min_child_samples', 20,100,1),
    'reg_alpha': hp.uniform('reg_alpha', 0.01, 0.9),
    'reg_lambda': hp.uniform('reg_lambda', 0.01, 0.9),    
}

In [ ]:
def objective(params, n_folds, x, y):
    
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    params['min_samples_split'] = int(params['min_samples_split'])
    params['max_leaf_nodes'] = int(params['max_leaf_nodes'])
    
    
    
    model = lgb.LGBMClassifier(**params, random_state = 0)
    
    scores = cross_val_score(model, x, y, cv = n_folds, scoring = 'roc_auc', error_score='raise')
    
    max_score = max(scores)
    
    # to minimize
    loss = 1 - max_score
    
    return {'loss':loss,
           'params':params,
           'status': STATUS_OK}

In [ ]:
# optimize with the TPE algorithm
trials = Trials()
n_folds = 10

best = fmin(fn = partial(objective, n_folds = n_folds,
                         x = xtrain_exp1, y = ytrain_exp1),
           space = search_space, algo = tpe.suggest, max_evals = 16, trials = trials,
           )

In [ ]:
print(best)

## Preprocessor - Experiment 2

1. Missing Values: Percentile Imputation
2. Encoding: Ordinal
3. Transformation: Sqrt
4. Outliers: Winsorize(0.95-0.05)
5. Scaling: RobustScaler
6. Resampling: ADASYN
7. Feature Selection: SelectFromModel

In [ ]:
categorical_exp2, numerical_exp2, xtrain_exp2, ytrain_exp2 = split_data_by_type(full_train_df, config['EDA']['target'])

categorical_exp2, numerical_exp2, xvalidation_exp2, yvalidation_exp2 = split_data_by_type(validation_df, config['EDA']['target'])

In [ ]:
num_pipeline_exp2 = Pipeline([
    ('transformer', FunctionTransformer(np.sqrt)),
    ('imputer', FunctionTransformer(percentile_imputer)),
    ('winsorize', FunctionTransformer(winsorize)),
    ('scaler', RobustScaler()),  
])

cat_pipeline_exp2 = Pipeline([
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('encoder', OrdinalEncoder()),
    
])

In [ ]:
pipeline_exp2 = create_pipeline(num_pipeline_exp2, numerical_exp2, cat_pipeline_exp2, categorical_exp2)

In [ ]:
xtrain_exp2 = pipeline_exp2.fit_transform(xtrain_exp2)

In [ ]:
xvalidation_exp2 = pipeline_exp2.transform(xvalidation_exp2)

In [ ]:
# adasyn

adasyn = ADASYN()

In [ ]:
xtrain_exp2, ytrain_exp2 = adasyn.fit_resample(xtrain_exp2, ytrain_exp2)

In [ ]:
rf = RandomForestClassifier()

selector = SelectFromModel(rf, threshold=0.0001).fit(xtrain_exp2, ytrain_exp2)

In [ ]:
selected_columns_exp2 = xtrain_exp2.columns[selector.get_support()]

len(selected_columns_exp2)

In [ ]:
xtrain_exp2 = xtrain_exp2[selected_columns_exp2]

xvalidation_exp2 = xvalidation_exp2[selected_columns_exp2]

### Hyperparameter Tuning - Exp 2

#### Random Forest

In [ ]:
# create the search space for an over fitting model

search_space = {
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'criterion': hp.choice('criterion', ['gini', 'entropy']),
    'max_depth': hp.quniform('max_depth', 2, 30, 1),
    'min_samples_split': hp.quniform('min_samples_split', 4, 30, 1),
    'min_weight_fraction_leaf': hp.uniform('min_weight_fraction_leaf', 0.0, 0.5),
    'max_leaf_nodes': hp.quniform('max_leaf_nodes',8,30,1),
    'warm_start': hp.choice('warm_start', [True, False]),
    'class_weight': hp.choice('class_weight', ['balanced', 'balanced_subsample']),
    'ccp_alpha': hp.uniform('ccp_alpha', 0.0, 0.9)
   
}

In [ ]:
def objective(params, n_folds, x, y):
    
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    params['min_samples_split'] = int(params['min_samples_split'])
    params['max_leaf_nodes'] = int(params['max_leaf_nodes'])
    
#     params['min_weight_fraction_leaf'] = round(params['min_weight_fraction_leaf'], 2)
#     params['ccp_alpha'] = round(params['ccp_alpha'], 2)
    
    
    model = RandomForestClassifier(**params, random_state = 0)
    
    scores = cross_val_score(model, x, y, cv = n_folds, scoring = 'roc_auc', error_score='raise')
    
    max_score = max(scores)
    
    # to minimize
    loss = 1 - max_score
    
    return {'loss':loss,
           'params':params,
           'status': STATUS_OK}

In [ ]:
# optimize with the TPE algorithm
trials = Trials()
n_folds = 10

best = fmin(fn = partial(objective, n_folds = n_folds,
                         x = xtrain_exp1, y = ytrain_exp1),
           space = search_space, algo = tpe.suggest, max_evals = 16, trials = trials,
           )